## conclude answer

* DataRobotで生成した一次回答から本回答を生成する

In [1]:
!pip install python-dotenv
!pip install pandas
!pip install langchain-core==0.3.0
!pip install langchain-openai==0.2.0
!pip install tiktoken


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
load_dotenv(verbose=True)
dotenv_path = ".env"
load_dotenv(dotenv_path)

import pandas as pd
pd.set_option('display.max_rows', 500)

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

import tiktoken

### データロード

In [3]:
df = pd.read_csv("../data/predictions/pre_predictions.csv", index_col=0)
df

,problem,company_name,reason,answer
index,,,,
0,高松コンストラクショングループの2025年3月期の受注高の計画は前期比何倍か、小数第三位を四...,株式会社高松コンストラクショングループ,高松コンストラクショングループの受注高の計画に関する情報を参照する必要があるため。,"2025年3月期の受注高の計画は3,900億円です。2024年3月期の実績は3,259億円で..."
1,株式会社キッツの取締役の報酬のうち株式報酬の割合は何％？,株式会社キッツ,ユーザが保有している企業レポートに株式会社キッツが含まれているため,取締役の報酬における株式報酬の割合は20%です。
2,グローリーが2024年の統合報告書の中で、研究開発費が増加した理由として挙げている事由は何？,グローリー株式会社,ユーザが保有しているレポートの中にグローリー株式会社の企業レポートが含まれているため,グローリーが2024年の統合報告書で研究開発費が増加した理由として挙げているのは、研究開発費...
3,高松コンストラクショングループの2024年3月期の非常勤監査役は何人？,株式会社高松コンストラクショングループ,ユーザが保有している企業レポートに該当する情報が含まれているため,高松コンストラクショングループにおける2024年3月期の非常勤監査役の人数は、表に記載されて...
4,2023年で即席めんの一人当たりの年間消費量が最も多い国はどこか。,日清食品ホールディングス株式会社,日清食品は即席めんの主要なメーカーであり、消費量に関する情報を持っている可能性が高いから。,提供された情報には、2023年における即席めんの一人当たりの年間消費量に関するデータは含まれ...
5,株式会社キッツで2023年度のデジタル教育受講者数は何人？,株式会社キッツ,ユーザが保有している企業レポートの中に株式会社キッツに関する情報が含まれているため。,2023年度のデジタル教育受講者数は、活動開始から2年で4.5倍以上に増加したと記載されてい...
6,2023年度の日清食品ホールディングスの海外事業において、コア営業利益が2番目に高い地域に含...,日清食品ホールディングス株式会社,2023年度の日清食品ホールディングスの海外事業に関する情報を提供するため。,申し訳ありませんが、提供された情報には日清食品ホールディングスの海外事業に関する具体的なデー...
7,株式会社キッツの2023年度売上高構成比が2番目に高い事業の製造・販売会社を全て上げよ。（回...,株式会社キッツ,2023年度売上高構成比に関する情報を得るために株式会社キッツのレポートを参照する必要があるため。,株式会社キッツの2023年度の売上高構成比が2番目に高い事業は「伸銅品事業」で、売上高構成比...
8,IHIの2023年度の資源・エネルギー・環境事業領域の営業利益が過去最高を記録した要因として...,株式会社IHI,ユーザが保有している企業レポートの中に株式会社IHIのレポートがあるため。,IHIの2023年度の資源・エネルギー・環境事業領域における営業利益が過去最高を記録した要因...


### 一次回答から本回答を生成

In [4]:
class Conclusion(BaseModel):
    conlusion: str = Field(description="質問に対して、クイズに答えるように簡潔で端的に修正した回答")

In [5]:
def conclude(query, answer):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "user",
                [
                    {
                        "type": "text",
                        "text": (
                            "ある質問に対して、AIが生成した回答があります。\n"
                            "元の回答が冗長のため、クイズに答えるように簡潔で端的な回答に修正してください。\n"
                            "元の回答が明確でない場合、「わかりません」と回答を修正してください。\n\n"
                            "質問:\n"
                            "{query}\n\n"
                            "回答:\n"
                            "{answer}"
                        ),
                    },
                ]
            )
        ]
    )

    model = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
    )

    chain = prompt | model.with_structured_output(Conclusion)

    conclusion = chain.invoke({
        "query": query,
        "answer": answer
    })

    return conclusion

テスト

In [6]:
conclude(
    "株式会社キッツの取締役の報酬のうち株式報酬の割合は何％？",
    "取締役の報酬における株式報酬の割合は20%です。",
)

Conclusion(conlusion='20%')

In [7]:
conclude(
    "2023年で即席めんの一人当たりの年間消費量が最も多い国はどこか。",
    "日清食品は即席めんの主要なメーカーであり、消費量に関する情報を持っている可能性が高いから。,提供された情報には、2023年における即席めんの一人当たりの年間消費量に関するデータは含まれていません。そのため、正確な回答をすることができません。",
)

Conclusion(conlusion='わかりません。')

In [8]:
conclude(
    "IHIグループの経営目標の進捗の2022年度と2023年度を比較したとき、営業利益率と税引後ROICではどちらの方が差が大きいか。",
    (
        "IHIグループの2022年度と2023年度の営業利益率と税引後ROICのデータは以下の通りです。\n\n"
        "- **2022年度**\n"
        "  - 営業利益率: 6.1%\n"
        "  - 税引後ROIC: 11.0%\n\n"
        "- **2023年度**\n"
        "  - 営業利益率: 5.3%\n"
        "  - 税引後ROIC: 16.9%\n\n"
        "次に、各年度の指標の差を計算します。\n\n"
        "1. **営業利益率の差**\n"
        "   - 2022年度 - 2023年度 = 6.1% - 5.3% = 0.8%\n\n"
        "2. **税引後ROICの差**\n"
        "   - 2023年度 - 2022年度 = 16.9% - 11.0% = 5.9%\n\n"
        "この結果から、税引後ROICの差（5.9%）の方が営業利益率の差（0.8%）よりも大きいことがわかります。したがって、税引後ROICの差が大きいです。"
    ),
)

Conclusion(conlusion='税引後ROICの差が大きい。')

本処理

In [9]:
def conclude_column(row):
    conclusion = conclude(row["problem"], row["answer"])
    
    return conclusion.conlusion

In [10]:
df["conclusion"] = df.apply(conclude_column, axis=1)
df

,problem,company_name,reason,answer,conclusion
index,,,,,
0,高松コンストラクショングループの2025年3月期の受注高の計画は前期比何倍か、小数第三位を四...,株式会社高松コンストラクショングループ,高松コンストラクショングループの受注高の計画に関する情報を参照する必要があるため。,"2025年3月期の受注高の計画は3,900億円です。2024年3月期の実績は3,259億円で...",1.196
1,株式会社キッツの取締役の報酬のうち株式報酬の割合は何％？,株式会社キッツ,ユーザが保有している企業レポートに株式会社キッツが含まれているため,取締役の報酬における株式報酬の割合は20%です。,20%
2,グローリーが2024年の統合報告書の中で、研究開発費が増加した理由として挙げている事由は何？,グローリー株式会社,ユーザが保有しているレポートの中にグローリー株式会社の企業レポートが含まれているため,グローリーが2024年の統合報告書で研究開発費が増加した理由として挙げているのは、研究開発費...,研究開発費の計上基準を変更したため。
3,高松コンストラクショングループの2024年3月期の非常勤監査役は何人？,株式会社高松コンストラクショングループ,ユーザが保有している企業レポートに該当する情報が含まれているため,高松コンストラクショングループにおける2024年3月期の非常勤監査役の人数は、表に記載されて...,3人
4,2023年で即席めんの一人当たりの年間消費量が最も多い国はどこか。,日清食品ホールディングス株式会社,日清食品は即席めんの主要なメーカーであり、消費量に関する情報を持っている可能性が高いから。,提供された情報には、2023年における即席めんの一人当たりの年間消費量に関するデータは含まれ...,わかりません
5,株式会社キッツで2023年度のデジタル教育受講者数は何人？,株式会社キッツ,ユーザが保有している企業レポートの中に株式会社キッツに関する情報が含まれているため。,2023年度のデジタル教育受講者数は、活動開始から2年で4.5倍以上に増加したと記載されてい...,わかりません
6,2023年度の日清食品ホールディングスの海外事業において、コア営業利益が2番目に高い地域に含...,日清食品ホールディングス株式会社,2023年度の日清食品ホールディングスの海外事業に関する情報を提供するため。,申し訳ありませんが、提供された情報には日清食品ホールディングスの海外事業に関する具体的なデー...,わかりません。
7,株式会社キッツの2023年度売上高構成比が2番目に高い事業の製造・販売会社を全て上げよ。（回...,株式会社キッツ,2023年度売上高構成比に関する情報を得るために株式会社キッツのレポートを参照する必要があるため。,株式会社キッツの2023年度の売上高構成比が2番目に高い事業は「伸銅品事業」で、売上高構成比...,株式会社キッツメタルワークス、株式会社清水合金製作所
8,IHIの2023年度の資源・エネルギー・環境事業領域の営業利益が過去最高を記録した要因として...,株式会社IHI,ユーザが保有している企業レポートの中に株式会社IHIのレポートがあるため。,IHIの2023年度の資源・エネルギー・環境事業領域における営業利益が過去最高を記録した要因...,事業ポートフォリオの見直し、ライフサイクルビジネスの拡大、新規事業の創出、技術力の強化。


トークン数チェック

In [11]:
def count_tokens(row):
    encoding = tiktoken.encoding_for_model("gpt-4o-mini")
    tokens = encoding.encode(row["conclusion"])
    num_tokens = len(tokens)

    return num_tokens

In [12]:
df["num_tokens"] = df.apply(count_tokens, axis=1)

In [13]:
df[df["num_tokens"]>55]

,problem,company_name,reason,answer,conclusion,num_tokens
index,,,,,,
27,カゴメが人材育成において重要視している観点を全てあげてください。,カゴメ株式会社,カゴメ株式会社の人材育成に関する情報を参照する必要があるため。,カゴメが人材育成において重要視している観点は以下の通りです。\n\n1. **多様な強みの伸...,多様な強みの伸長、イノベーションの創出、自律的な成長の促進、ダイバーシティ&インクルージョン...,68


In [14]:
df.loc[df["num_tokens"]>55, "conclusion"] = "わかりません"

保存

In [15]:
df_predictions = df[["conclusion"]]
df_predictions.to_csv("../data/predictions/predictions.csv", header=False)